In [1]:
import re
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from preprocess import get_records, preprocess_for_logistic, read_records


In [1]:
import re
import os
import pandas as pd
import numpy as np
import tensorflow as tf
#from preprocess import get_records, preprocess_for_logistic, read_records

# read and preprocess training and validation dataset for average pool, validation dataset

In [3]:
r = re.compile("^train.+\\.tfrecord$")
train_dir  = "/Users/shufanxia/Documents/IDS705/Video_Popularity_Prediction/frame-level/"
val_dir = "/Users/shufanxia/Documents/IDS705/Video_Popularity_Prediction/validation-frame/"

frames_train = get_records(train_dir,"train")
frames_val = get_records(val_dir,"validate")

n_labels = 300
feat_rgb,feat_audio,feat_puesdoid,feat_labels = read_records(frames_train[:1])
X_rgb_train, X_audio_train,y_train = preprocess_for_logistic(feat_rgb,feat_audio,feat_labels,n_labels)

feat_rgb_val,feat_audio_val,feat_puesdoid_val,feat_labels_val = read_records(frames_val[:1])
X_rgb_val, X_audio_val,y_val = preprocess_for_logistic(feat_rgb_val,feat_audio_val,feat_labels_val,n_labels)

# Baseline logsitic model

In [4]:

def prepare_logistic(input_type = "rgb", l2= 1e-8,X_rgb_train=None,X_audio_train = None, y_train=None,
                     X_rgb_val=None,X_audio_val= None,y_val=None):
    # build and train a one vs all multiclass classifier
    # choose from three types of inputs
    if input_type == "rgb":
        X_train = X_rgb_train
        X_val = X_rgb_val
    elif input_type == "audio":
        X_train = X_audio_train
        X_val = X_audio_val
    elif input_type == "both":
        X_train = tf.concat([X_rgb_train, X_audio_train],1)
        X_val = tf.concat([X_rgb_val, X_audio_val],1)
    else:
        print("invalid input type")
        raise ValueError
    print(len(X_train))
    logistic_reg = tf.keras.models.Sequential([
                tf.keras.layers.Dense(n_labels, activation='sigmoid',
                            kernel_regularizer=tf.keras.regularizers.L2(l2))
                ])

    optimizer = tf.keras.optimizers.SGD()
    logistic_reg.compile(optimizer,
                loss=tf.keras.losses.BinaryCrossentropy())
    # one vs all multiclass classifier, print binary loss along the way

    # randomzie befor training
    indices = tf.range(start=0, limit=tf.shape(X_train)[0], dtype=tf.int32)
    # train, print training loss and validation loss after each epoch
    logistic_reg.fit(X_train,y_train,epochs=10,
                    batch_size=500,
                    validation_data = (X_val,y_val))

    return X_train,X_val,y_train,y_val, logistic_reg

##### With just rgb

In [5]:
X_train,X_val,y_train,y_val, logistic_reg = prepare_logistic(input_type = "rgb",l2= 1e-8,
                                                X_rgb_train = X_rgb_train,y_train =y_train, 
                                                X_rgb_val=X_rgb_val, y_val=y_val)
logistic_reg.fit(X_train,y_train,epochs=10,
                    batch_size=500,
                    validation_data = (X_val,y_val))

1015
Epoch 1/10
3/3 [==============================] - 0s 49ms/step - loss: 33.8826 - val_loss: 2.0219
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 1.9397 - val_loss: 1.5565
Epoch 3/10
3/3 [==============================] - 0s 15ms/step - loss: 1.5508 - val_loss: 1.4893
Epoch 4/10
3/3 [==============================] - 0s 14ms/step - loss: 1.5076 - val_loss: 1.4590
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 1.3787 - val_loss: 1.5467
Epoch 6/10
3/3 [==============================] - 0s 14ms/step - loss: 1.4384 - val_loss: 1.4349
Epoch 7/10
3/3 [==============================] - 0s 13ms/step - loss: 1.3309 - val_loss: 1.7304
Epoch 8/10
3/3 [==============================] - 0s 14ms/step - loss: 1.5595 - val_loss: 1.3840
Epoch 9/10
3/3 [==============================] - 0s 14ms/step - loss: 1.3546 - val_loss: 1.3481
Epoch 10/10
3/3 [==============================] - 0s 14ms/step - loss: 1.2673 - val_loss: 1.4493
Epoch 1/10
3/3 [=======

##### With just audio

In [27]:
X_train,X_val,y_train,y_val, logistic_reg = prepare_logistic(input_type = "audio",l2= 1e-8,
                                                X_audio_train =X_audio_train,y_train =y_train, 
                                                X_audio_val= X_audio_val,y_val=y_val)
logistic_reg.fit(X_train,y_train,epochs=10,
                    batch_size=500,
                    validation_data = (X_val,y_val))

here
1015
Epoch 1/10
3/3 [==============================] - 0s 38ms/step - loss: 30.1254 - val_loss: 2.8566
Epoch 2/10
3/3 [==============================] - 0s 9ms/step - loss: 2.3450 - val_loss: 1.0727
Epoch 3/10
3/3 [==============================] - 0s 11ms/step - loss: 1.1485 - val_loss: 0.9241
Epoch 4/10
3/3 [==============================] - 0s 10ms/step - loss: 1.0021 - val_loss: 0.8496
Epoch 5/10
3/3 [==============================] - 0s 10ms/step - loss: 0.9250 - val_loss: 0.8081
Epoch 6/10
3/3 [==============================] - 0s 11ms/step - loss: 0.8797 - val_loss: 0.7853
Epoch 7/10
3/3 [==============================] - 0s 12ms/step - loss: 0.8514 - val_loss: 0.7698
Epoch 8/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8296 - val_loss: 0.7550
Epoch 9/10
3/3 [==============================] - 0s 9ms/step - loss: 0.8098 - val_loss: 0.7580
Epoch 10/10
3/3 [==============================] - 0s 10ms/step - loss: 0.8013 - val_loss: 0.7353
Epoch 1/10
3/3 [====

#### With video + audio

In [29]:

X_train,X_val,y_train,y_val, logistic_reg = prepare_logistic(input_type = "both",l2= 1e-8,
                                                        X_rgb_train = X_rgb_train,X_audio_train =X_audio_train, y_train =y_train,  
                                                            X_rgb_val=X_rgb_val, X_audio_val=X_audio_val,  y_val=y_val,)
logistic_reg.fit(X_train,y_train,epochs=10,
                    batch_size=500,
                    validation_data = (X_val,y_val))

1015
Epoch 1/10
3/3 [==============================] - 0s 37ms/step - loss: 23.9635 - val_loss: 1.8920
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 1.9380 - val_loss: 1.8908
Epoch 3/10
3/3 [==============================] - 0s 15ms/step - loss: 1.9046 - val_loss: 1.8283
Epoch 4/10
3/3 [==============================] - 0s 14ms/step - loss: 1.7635 - val_loss: 1.7190
Epoch 5/10
3/3 [==============================] - 0s 15ms/step - loss: 1.6806 - val_loss: 1.6464
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 1.6094 - val_loss: 1.8054
Epoch 7/10
3/3 [==============================] - 0s 14ms/step - loss: 1.7806 - val_loss: 1.5585
Epoch 8/10
3/3 [==============================] - 0s 14ms/step - loss: 1.6333 - val_loss: 1.4521
Epoch 9/10
3/3 [==============================] - 0s 14ms/step - loss: 1.4002 - val_loss: 1.6514
Epoch 10/10
3/3 [==============================] - 0s 13ms/step - loss: 1.6352 - val_loss: 1.4508
Epoch 1/10
3/3 [=======

In [ ]:
print(tf. __version__) 

2.7.0


In [14]:
import eval_util as eval


y_predproba_train = logistic_reg.predict(X_train)
y_train2  = tf.cast(y_train,tf.int32)
gAP_train = eval.calculate_gap(y_predproba_train ,y_train2)
PERR_train = eval.calculate_precision_at_equal_recall_rate(y_predproba_train ,y_train)
# HIT1_train= eval.calculate_hit_at_one(y_predproba_train ,y_train)
# # f1 score
# print("Training performance: gAP = %.3f, PERR = %.3f, HIT1 = %.3f"%(gAP_train,PERR_train, HIT1_train))

# y_predproba_val = logistic_reg.predict(X_val)
# #gAP_val = eval.calculate_gap(y_predproba_val ,y_val)
# #PERR_val = eval.calculate_precision_at_equal_recall_rate(y_predproba_val ,y_val)
# HIT1_val= eval.calculate_hit_at_one(y_predproba_val ,y_val)
# # f1 score
# print("Validation performance: gAP = %.3f, PERR = %.3f, HIT1 = %.3f"%(gAP_val,PERR_val, HIT1_val))

TypeError: Cannot convert 0.0 to EagerTensor of dtype bool

In [34]:
y_predproba_val.shape

(322, 300)

In [37]:
tensor.shape

TensorShape([322, 300])

In [36]:

tensor = tf.cast(y_val, tf.bool)
HIT1_val= eval.calculate_hit_at_one(y_predproba_val ,tensor)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

In [24]:
label_dict = pd.read_csv("vocabulary.csv")

def get_label(label_index,top_n_labels):
  def get_label_row(row):
    if type(row)!= np.ndarray:
      row= np.array(row)
    labels = label_dict.iloc[row[row<top_n_labels]].Name.values
    return labels

  all_labels  = list(map(lambda row: get_label_row(row), label_index))
  return all_labels

def make_top_n_pred_df(pesudo_id,y_predproba,feat_labels,top_n = 5):
  top_labels_proba = np.fliplr(np.sort(y_predproba,axis=1))[:,:top_n]
  top_labels_pred = np.fliplr(y_predproba.argsort(axis=1))[:,:top_n]
  label_true = get_label(feat_labels,1000)
  label_pred = get_label(top_labels_pred,1000)
  predict_df = pd.DataFrame({"pesudo_id":pesudo_id,
              "label_true":label_true,
              "label_pred":label_pred,
              "predict_proba":top_labels_proba.tolist()})
  return predict_df


In [28]:
train_pred_df = make_top_n_pred_df(feat_puesdoid,y_predproba_train,feat_labels,top_n = 5,get_names=True)
train_pred_df.head(10)


,pesudo_id,label_true,label_pred,predict_proba
0,lKbF,"[Video game, Racing]","[Drum, Musician, Guitar, Choir, Game]","[1.0, 1.0, 1.0, 0.9999840259552002, 0.98466122..."
1,HebF,"[Concert, Performance art]","[Musician, Guitar, Dance, Acoustic guitar, Mob...","[1.0, 1.0, 0.008513122797012329, 1.11282688860..."
2,GkbF,[Fashion],"[Musician, Guitar, Acoustic guitar, Associatio...","[1.0, 1.0, 3.8150992622831836e-05, 1.131936855..."
3,q2bF,[Music video],"[Musician, Guitar, Fighting game, Acoustic gui...","[1.0, 0.9999983310699463, 0.09486424922943115,..."
4,aUbF,"[RuneScape, Beach, Sneakers]","[Musician, Game, Dance, Acoustic guitar, Guitar]","[1.0, 0.9147140979766846, 0.6104596853256226, ..."
5,A9bF,"[Vehicle, Motorsport, Piano, Transport, BMW]","[Guitar, Musician, Association football, Fight...","[0.9999978542327881, 0.9999697804450989, 1.657..."
6,52bF,[Trailer (promotion)],"[Musician, Acoustic guitar, Guitar, Dance, Orc...","[1.0, 0.9999675750732422, 7.4926992965629324e-..."
7,gNbF,"[Vehicle, Car]","[Musician, Mobile phone, Acoustic guitar, Guit...","[1.0, 0.018159449100494385, 5.379041340347612e..."
8,6ZbF,"[Video game, Television, DayZ (video game)]","[Acoustic guitar, Musician, Guitar, Mobile pho...","[1.0, 1.0, 1.0, 0.14578327536582947, 2.8150027..."
9,rubF,"[Cosmetics, Fashion]","[Acoustic guitar, Musician, Guitar, First-pers...","[1.0, 1.0, 0.9846801161766052, 2.8657879624915..."


In [31]:
# training data
val_pred_df = make_top_n_pred_df(feat_puesdoid_val,y_predproba_val,feat_labels_val,top_n = 5,get_names=False)
val_pred_df.head(10)

,pesudo_id,label_true,label_pred,predict_proba
0,Buay,"[Game, Video game, Sports game, Extreme sport]","[Guitar, Musician, Mobile phone, Piano, Perfor...","[1.0, 1.0, 0.3430946469306946, 3.3650237583060..."
1,Jeay,"[Game, Association football, Highlight film]","[Musician, Acoustic guitar, Guitar, Dance, Sma...","[1.0, 0.0006382465362548828, 0.000107368359749..."
2,Geay,"[Vehicle, Car, Driving]","[Musician, Dance, Guitar, Association football...","[1.0, 0.9999986886978149, 0.9998586177825928, ..."
3,lSay,[Concert],"[Sedan (automobile), Rail transport, Musician,...","[1.0, 1.0, 1.0, 1.0, 0.9999985694885254]"
4,JMay,"[Musician, Guitar, String instrument, Acoustic...","[Musician, Acoustic guitar, Guitar, Electric g...","[1.0, 1.0, 1.0, 0.9999984502792358, 7.50024575..."
5,vEay,[Rail freight transport],"[Guitar, Musician, Super Smash Bros., Railroad...","[1.0, 1.0, 0.9990826845169067, 0.8304609656333..."
6,Nday,"[Game, Association football, Highlight film]","[Musician, Guitar, Game, Acoustic guitar, Smar...","[1.0, 0.9999995827674866, 0.899193525314331, 4..."
7,XBay,"[Food, Recipe, Cooking, Cuisine, Dish (food), ...","[Musician, Smartphone, Acoustic guitar, Mobile...","[1.0, 5.69418534723809e-06, 1.4456688859354472..."
8,wgay,"[Pet, Dog, Puppy, Sewing]","[Musician, Guitar, Smartphone, Piano, Mobile p...","[1.0, 0.9999997615814209, 8.048542440519668e-0..."
9,1say,"[Food, Disc jockey, Wheelie, Loader (equipment)]","[Musician, Guitar, Toy, Food, Fighting game]","[1.0, 1.0, 2.2412124933701705e-10, 5.725077009..."
